# SQL

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain import OpenAI, SelfAskWithSearchChain, LLMMathChain
from sqlalchemy.engine import Engine, create_engine
from langchain.agents import initialize_agent, Tool
import os

In [3]:
import sys
 # setting path
sys.path.append('..')
sys.path.append('../modules')
sys.path.append('../modules/database/')

from modules.database.sql_chain import SQLDatabaseChain
from modules.database.sql_database import SQLDatabase

In [4]:
# connect to remote postgres database
db = SQLDatabase.from_uri(os.getenv("DB_CONNECTION_STRING"))
print("Tables found:", db._all_tables)
db_chain = SQLDatabaseChain(llm=OpenAI(), database=db, verbose=True, debug=True)

Tables found: ['users', 'templates', 'models', 'generations']


In [5]:
# Load the tool configs that are needed.
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Database",
        func=db_chain.run,
        description="useful for when you need to answer questions about data in your database"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [6]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [7]:
agent.run("We had a bug yesterday. Are there any users that are stuck in training?")



> Entering new ZeroShotAgent chain...
We had a bug yesterday. Are there any users that are stuck in training?
Thought: I need to check the database to see if any users are stuck in training.
Action: Database
Action Input: Check for users stuck in training

> Entering new SQLDatabaseChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

Table 'users' has columns: id (UUID), email (TEXT), payment_status (TEXT), ios_id (TEXT), fcm_token (TEXT), credits (BIGINT), finetune_access (BOOLEAN), experiment (TEXT).
Table 'generations' has columns: id (UUID), created_at (TIMESTAMP WITH TIME ZONE), updated_at (TIMESTAMP WITH TIME ZONE), user_id (UUID), inp

'No, there are no users stuck in training.'